### MySQL 접속용 샘플 프로그램

##### 1. 접속 정보가 들어 있는 파일 읽기

In [24]:
import json

with open('./mysql.json') as f:
    config_str = f.read()

config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'ysuser',
 'password': 'yspass',
 'database': 'world',
 'port': 3306}

##### 2. MySQL에 접속

In [3]:
import pymysql
conn = pymysql.connect(
    host=config['host'], user=config['user'], password=config['password'], 
    database=config['database'], port=config['port']
)

##### 3. MySQL Access

- city table에서 10개 데이터 읽기

In [4]:

cur = conn.cursor()
sql = "SELECT * FROM city LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
cur.close()

for row in rows:
    print(row)


(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)


In [6]:
import pandas as pd
df = pd.DataFrame(rows, columns=['id', 'name', 'country_code', 'district', 'population'])
df.set_index('id', inplace=True)
df

,name,country_code,district,population
id,,,,
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900
8,Utrecht,NLD,Utrecht,234323
9,Eindhoven,NLD,Noord-Brabant,201843


- 검색: 국가코드 KOR 의 인구수 Top 10

In [11]:
# 데이터 여러건 가져오기
cur = conn.cursor()
sql = "SELECT * FROM city 	WHERE countrycode=%s ORDER BY population DESC LIMIT 10;" # %s로 전달
cur.execute(sql, ('KOR', ))     # 파라메터는 반드시 튜플로 전달해야 함(주의: 한개더라도)
rows = cur.fetchall()
cur.close()

In [12]:
for row in rows:
    print(row)

(2331, 'Seoul', 'KOR', 'Seoul', 10000000)
(5006, '대도시', 'KOR', 'Kyonggi', 8000000)
(2332, 'Pusan', 'KOR', 'Pusan', 3804522)
(2333, 'Inchon', 'KOR', 'Inchon', 2559424)
(2334, 'Taegu', 'KOR', 'Taegu', 2548568)
(2335, 'Taejon', 'KOR', 'Taejon', 1425835)
(2336, 'Kwangju', 'KOR', 'Kwangju', 1368341)
(2340, 'Suwon', 'KOR', 'Kyonggi', 1200000)
(2337, 'Ulsan', 'KOR', 'Kyongsangnam', 1084891)
(2338, 'Songnam', 'KOR', 'Kyonggi', 869094)


In [21]:
# 데이터 한 건 가져오기
cur = conn.cursor()
sql = "SELECT * FROM city 	WHERE name=%s" 
cur.execute(sql, ('Suwon', )) 
row = cur.fetchone()
cur.close()
row

(2340, 'Suwon', 'KOR', 'Kyonggi', 1200000)

- 데이터 추가(Insert)

In [14]:
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, '자바', 'KOR', 'Kyonggi', 6000000)"
cur.execute(sql)
conn.commit()       # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()


In [15]:
# 데이터 1건 insert
params = ('노드JS', 'KOR', 'Kyonggi', 6000000)
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"
cur.execute(sql, params)
conn.commit()
cur.close()

In [16]:
# 데이터 여러건을 for로 한건씩 insert
params_list = [
    ('html', 'KOR', 'Kyonggi', 7000000), 
    ('css', 'KOR', 'Kyonggi', 7200000)
]
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"
for params in params_list:
    cur.execute(sql, params)
conn.commit()
cur.close()

In [17]:
# 데이터 한번에 insert
params_list = [
    ('html1', 'KOR', 'Kyonggi', 7000000), 
    ('css1', 'KOR', 'Kyonggi', 7200000)
]
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"
cur.executemany(sql, params_list)       # 몇 건은 가능하나 대량은 돌아가지 않을 수 있다. for문으로 추천한다.
conn.commit()
cur.close()

- 데이터 수정(Update)

In [18]:
cur = conn.cursor()
sql = "UPDATE city SET name='CSS10' WHERE id=5011;"
cur.execute(sql)
conn.commit()
cur.close()

In [19]:
params_list = [('html5', 5011), ('html5', 5013)]
cur = conn.cursor()
sql = "UPDATE city SET name=%s WHERE id=%s;"
cur.executemany(sql, params_list)
conn.commit()
cur.close()

- 데이터 삭제(Delete)

In [20]:
cur = conn.cursor()
sql = "DELETE FROM city WHERE id > %s;"
cur.execute(sql, (5000, ))
conn.commit()
cur.close()

##### 4. MySQL 접속 해제

In [22]:
conn.close()